In [69]:
!which python

/home/faske/g/anaconda3/envs/py36/bin/python


In [70]:
#!source activate py34
!source activate py36

In [71]:
import sys

#sys.path.append('/home/faske/g/anaconda3/envs/py34/lib/python3.4/site-packages')
sys.path.append('/home/faske/g/anaconda3/envs/py36/lib/python3.6/site-packages')
sys.path.append("/home/faske/g/ipynb/include_utils")

import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
#import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE, call, check_output
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
#import pysam
from collections import OrderedDict, namedtuple, Counter
import operator
import multiprocessing as mp
import shutil
import tempfile
#from ipyparallel import Client
import scandir
import glob
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import pickle
#import Levenshtein as lv

In [72]:
vcfutils = "vcfutils.pl"
vcftools = "vcftools"
bcftools = "bcftools"
bgzip = "bgzip"
tabix = "tabix"
java = "java"
plink = "plink --noweb"

In [73]:
analysis_dir = '/home/faske/g/balsam/SNPcall/filtering'

In [74]:
cd $analysis_dir

/working/tfaske/balsam/SNPcall/filtering


In [75]:
vcf_file = os.path.join(analysis_dir, "samtools1-9.BASA.vcf.gz")
assert os.path.exists(vcf_file)
vcf_file

'/home/faske/g/balsam/SNPcall/filtering/samtools1-9.BASA.vcf.gz'

# Filter first round 
    missing data, MAF, biallelic

In [8]:
!$vcftools --remove-indels \
--max-missing 0.5 \
--min-alleles 2 \
--max-alleles 2 \
--maf 0.05 \
--thin 5 \
--remove-filtered-all \
--recode \
--recode-INFO-all \
--gzvcf \
$vcf_file \
--out $'BASA_miss50_thin5_MAF5'


VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf /home/faske/g/balsam/SNPcall/filtering/samtools1-9.BASA.vcf.gz
	--recode-INFO-all
	--maf 0.05
	--max-alleles 2
	--min-alleles 2
	--thin 5
	--max-missing 0.5
	--out BASA_miss50_thin5_MAF5
	--recode
	--remove-filtered-all
	--remove-indels

Using zlib version: 1.2.11
After filtering, kept 373 out of 373 Individuals
Outputting VCF file...
After filtering, kept 268810 out of a possible 1437211 Sites
Run Time = 557.00 seconds


In [76]:
!$vcftools --remove-indels \
--max-missing 0.7 \
--min-alleles 2 \
--max-alleles 2 \
--maf 0.05 \
--thin 100 \
--remove-filtered-all \
--recode \
--recode-INFO-all \
--gzvcf \
$vcf_file \
--out $'BASA_miss70_thin100_MAF5'


VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf /home/faske/g/balsam/SNPcall/filtering/samtools1-9.BASA.vcf.gz
	--recode-INFO-all
	--maf 0.05
	--max-alleles 2
	--min-alleles 2
	--thin 100
	--max-missing 0.7
	--out BASA_miss70_thin100_MAF5
	--recode
	--remove-filtered-all
	--remove-indels

Using zlib version: 1.2.11
After filtering, kept 373 out of 373 Individuals
Outputting VCF file...
After filtering, kept 65530 out of a possible 1437211 Sites
Run Time = 229.00 seconds


In [77]:
vcf_filtered = "%s.recode.vcf" % os.path.join(analysis_dir,'BASA_miss70_thin100_MAF5')
vcf_filtered_gz = "%s.gz" % vcf_filtered

In [78]:
!$bgzip -c $vcf_filtered > {vcf_filtered_gz}
!$tabix {vcf_filtered_gz}

# Remove bad_indv


In [79]:
!$vcftools --gzvcf $vcf_filtered_gz --out $vcf_filtered_gz --missing-indv


VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf /home/faske/g/balsam/SNPcall/filtering/BASA_miss70_thin100_MAF5.recode.vcf.gz
	--missing-indv
	--out /home/faske/g/balsam/SNPcall/filtering/BASA_miss70_thin100_MAF5.recode.vcf.gz

Using zlib version: 1.2.11
After filtering, kept 373 out of 373 Individuals
Outputting Individual Missingness
After filtering, kept 65530 out of a possible 65530 Sites
Run Time = 7.00 seconds


In [80]:
def get_imiss(filedir, prefix):
    imiss_file = !ls {filedir}/{prefix}.imiss
    imiss_df = pd.read_csv(imiss_file[0], sep="\t")
    imiss_df.index = imiss_df.INDV
    return imiss_df

In [81]:
imiss_df = get_imiss(analysis_dir,'BASA_miss70_thin100_MAF5.recode.vcf.gz')
imiss_df.head()

,INDV,N_DATA,N_GENOTYPES_FILTERED,N_MISS,F_MISS
INDV,,,,,
/home/faske/g/balsam/SNPcall/bowtie2/BS_AN_10.F.fq.gz_sorted.bam,/home/faske/g/balsam/SNPcall/bowtie2/BS_AN_10....,65530,0,4159,0.063467
/home/faske/g/balsam/SNPcall/bowtie2/BS_AN_11.F.fq.gz_sorted.bam,/home/faske/g/balsam/SNPcall/bowtie2/BS_AN_11....,65530,0,2960,0.045170
/home/faske/g/balsam/SNPcall/bowtie2/BS_AN_12.F.fq.gz_sorted.bam,/home/faske/g/balsam/SNPcall/bowtie2/BS_AN_12....,65530,0,4136,0.063116
/home/faske/g/balsam/SNPcall/bowtie2/BS_AN_1.F.fq.gz_sorted.bam,/home/faske/g/balsam/SNPcall/bowtie2/BS_AN_1.F...,65530,0,3755,0.057302
/home/faske/g/balsam/SNPcall/bowtie2/BS_AN_2.F.fq.gz_sorted.bam,/home/faske/g/balsam/SNPcall/bowtie2/BS_AN_2.F...,65530,0,2809,0.042866


In [82]:
imiss_df.F_MISS.describe()

count    373.000000
mean       0.089431
std        0.114146
min        0.014863
25%        0.054097
50%        0.066717
75%        0.089501
max        0.995880
Name: F_MISS, dtype: float64

In [83]:
len(imiss_df),len(imiss_df[imiss_df.F_MISS >= .25]),len(imiss_df[imiss_df.F_MISS >= .45]), len(imiss_df[imiss_df.F_MISS >= .25])


(373, 9, 6, 9)

In [84]:
bad_indv = imiss_df.INDV[imiss_df.F_MISS >= .25]

In [85]:
with open(os.path.join(analysis_dir, "bad_indv.txt"), "w") as o:
    o.write("INDV\n")
    for elem in bad_indv.index:
        o.write("%s\n" % elem)

In [86]:
!$vcftools --gzvcf $vcf_filtered_gz \
--remove-indels  \
--remove-filtered-all \
--recode \
--recode-INFO-all \
--remove {os.path.join(analysis_dir, "bad_indv.txt")} \
--out {os.path.join(analysis_dir, "snps_indv_removed")}


VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf /home/faske/g/balsam/SNPcall/filtering/BASA_miss70_thin100_MAF5.recode.vcf.gz
	--remove /home/faske/g/balsam/SNPcall/filtering/bad_indv.txt
	--recode-INFO-all
	--out /home/faske/g/balsam/SNPcall/filtering/snps_indv_removed
	--recode
	--remove-filtered-all
	--remove-indels

Using zlib version: 1.2.11
Excluding individuals in 'exclude' list
After filtering, kept 364 out of 373 Individuals
Outputting VCF file...
After filtering, kept 65530 out of a possible 65530 Sites
Run Time = 107.00 seconds


In [87]:
vcf_bad_remove = os.path.join(analysis_dir, "snps_indv_removed.recode.vcf")
vcf_bad_remove_gz = vcf_bad_remove + ".gz"
!$bgzip -c {vcf_bad_remove} > {vcf_bad_remove_gz}
!$tabix {vcf_bad_remove_gz}

# Filter snps further 
this needs to be done after removing individuals


In [88]:
def get_vcf_stats(vcf_gz):
    
    stats = ['depth',
            'site-depth',
            'site-mean-depth',
            'site-quality',
            'missing-site',
            'freq',
            'counts',
            'hardy',
            'het']
    
    for stat in stats:
        !$vcftools --gzvcf $vcf_gz \
        --out $vcf_gz \
        {"--%s" % stat} 

In [89]:
get_vcf_stats(vcf_bad_remove_gz)


VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf /home/faske/g/balsam/SNPcall/filtering/snps_indv_removed.recode.vcf.gz
	--depth
	--out /home/faske/g/balsam/SNPcall/filtering/snps_indv_removed.recode.vcf.gz

Using zlib version: 1.2.11
After filtering, kept 364 out of 364 Individuals
Outputting Mean Depth by Individual
After filtering, kept 65530 out of a possible 65530 Sites
Run Time = 7.00 seconds

VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf /home/faske/g/balsam/SNPcall/filtering/snps_indv_removed.recode.vcf.gz
	--out /home/faske/g/balsam/SNPcall/filtering/snps_indv_removed.recode.vcf.gz
	--site-depth

Using zlib version: 1.2.11
After filtering, kept 364 out of 364 Individuals
Outputting Depth for Each Site
After filtering, kept 65530 out of a possible 65530 Sites
Run Time = 8.00 seconds

VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf 

After filtering, kept 65530 out of a possible 65530 Sites
Run Time = 7.00 seconds


In [90]:
#pd.set_option('display.max_columns', 100)

def get_MAF(row):
    try:
        return np.min([row.A1_freq, row.A2_freq])
    except:
        print(row)
        
def get_correction(n):
    #for finite sample size
    return (2*n)/(2*n-1)

def calculate_Fis(vals):
    try:
        data = [float(x) for x in vals.split("/")]
        assert len(data) == 3
        num_individuals = np.sum(data)
        total_alleles = 2*num_individuals
        a1_count = 2*data[0]
        a2_count = 2*data[2]
        het_count = data[1]
        a1_count += het_count
        a2_count += het_count
        a1_freq = a1_count/total_alleles
        a2_freq = a2_count/total_alleles
        assert a1_freq + a2_freq == 1.0
        He = 2 * a1_freq * a2_freq * get_correction(num_individuals)
        Ho = het_count/num_individuals
        Fis = 1 - (Ho/He)
        return Fis
    except:
        return -9

def combine_vcf_stats(filedir, prefix):
    
    hardy_files = !ls {filedir}/{prefix}.hwe
    hardy = pd.read_csv(hardy_files[0], sep="\t")

    hardy.columns = ['CHROM', 'POS', 'OBS(HOM1/HET/HOM2)', 'E(HOM1/HET/HOM2)', 'ChiSq_HWE',
       'P_HWE', 'P_HET_DEFICIT', 'P_HET_EXCESS']
    hardy.index = hardy.apply(lambda x: "%s-%d" % (x.CHROM, x.POS), axis=1)
    
    loci_files = !ls {filedir}/{prefix}.l* | grep -v log
    loci_df = pd.concat([pd.read_csv(x, sep="\t", skiprows=0) for x in loci_files], axis=1)
    chrom_pos = loci_df.ix[:,0:2]
    
    frq_files = !ls {filedir}/{prefix}.frq* | grep -v count
    frq_data = []
    h = open(frq_files[0])
    header = h.readline().strip().split()
    for line in h:
        frq_data.append(line.strip().split('\t'))

    header = ['CHROM', 'POS', 'N_ALLELES', 'N_CHR', 'A1_FREQ', "A2_FREQ"]
    frq_df = pd.DataFrame(frq_data)
    print(frq_df.columns)
    #frq_df = frq_df.drop([6,7],axis=1)
    frq_df.columns = header
    frq_df.index = frq_df.apply(lambda x: "%s-%s" % (x.CHROM, x.POS), axis=1)
    
    loci_df = loci_df.drop(['CHROM','CHR','POS'], axis=1)
    loci_df = pd.concat([chrom_pos, loci_df], axis=1)
    loci_df.index = loci_df.apply(lambda x: "%s-%d" % (x.CHROM, x.POS), axis=1)
    
    loci_df = pd.concat([loci_df, frq_df, hardy], axis=1)
    loci_df["A1_allele"] = loci_df.apply(lambda row: row.A1_FREQ.split(":")[0], axis=1)
    loci_df["A2_allele"] = loci_df.apply(lambda row: row.A2_FREQ.split(":")[0], axis=1)
    
    loci_df["A1_freq"] = loci_df.apply(lambda row: float(row.A1_FREQ.split(":")[1]), axis=1)
    loci_df["A2_freq"] = loci_df.apply(lambda row: float(row.A2_FREQ.split(":")[1]), axis=1)
    
    loci_df['MAF'] = loci_df.apply(get_MAF, axis=1)
    loci_df = loci_df.drop(['CHROM', 'POS'], axis=1)
    
    loci_df['Fis'] = loci_df['OBS(HOM1/HET/HOM2)'].apply(calculate_Fis)
    
    return loci_df, frq_df, hardy

In [91]:
vcf_bad_remove_gz

'/home/faske/g/balsam/SNPcall/filtering/snps_indv_removed.recode.vcf.gz'

In [92]:
loci_df, frq_df, hardy = combine_vcf_stats(analysis_dir,'snps_indv_removed.recode.vcf.gz')

/home/faske/g/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


RangeIndex(start=0, stop=6, step=1)


In [93]:
loci_df

,SUM_DEPTH,SUMSQ_DEPTH,MEAN_DEPTH,VAR_DEPTH,N_DATA,N_GENOTYPE_FILTERED,N_MISS,F_MISS,QUAL,N_ALLELES,...,ChiSq_HWE,P_HWE,P_HET_DEFICIT,P_HET_EXCESS,A1_allele,A2_allele,A1_freq,A2_freq,MAF,Fis
dDocent_Contig_10-75,2891540,1495403936,7943.79000,-5.915810e+07,728,0,0,0.000000,999.0,2,...,65.945980,4.826685e-11,4.826685e-11,1.000000e+00,C,G,0.892857,0.107143,0.107143,0.426430
dDocent_Contig_20-67,2777103,163493983,7629.40000,-5.791780e+07,728,0,0,0.000000,999.0,2,...,330.997000,1.239780e-33,1.239780e-33,1.000000e+00,G,C,0.936813,0.063187,0.063187,0.953653
dDocent_Contig_1545-58,5157,89511,14.16760,4.531340e+01,728,0,0,0.000000,999.0,2,...,26.724590,3.078361e-08,1.000000e+00,1.612011e-08,G,T,0.255495,0.744505,0.255495,-0.269214
dDocent_Contig_1548-37,73317,19106115,201.42000,1.195200e+04,728,0,0,0.000000,999.0,2,...,3.266925,9.269385e-02,1.000000e+00,5.292059e-02,A,G,0.913462,0.086539,0.086539,-0.093233
dDocent_Contig_4618-31,17613,1244873,48.38740,1.081620e+03,728,0,0,0.000000,999.0,2,...,14.179360,1.169582e-05,1.000000e+00,7.427118e-06,G,A,0.835165,0.164835,0.164835,-0.195724
dDocent_Contig_4842-25,34938,4381622,95.98350,2.832370e+03,728,0,0,0.000000,999.0,2,...,0.308136,8.282703e-01,7.671565e-01,3.901890e-01,T,C,0.858516,0.141484,0.141484,-0.027682
dDocent_Contig_5506-50,2353,20065,6.46429,1.337340e+01,728,0,6,0.008242,999.0,2,...,15.162130,1.276071e-04,9.999706e-01,7.408596e-05,G,T,0.592798,0.407202,0.407202,-0.203271
dDocent_Contig_5693-58,2052,16728,5.63736,1.421520e+01,728,0,14,0.019231,999.0,2,...,7.326722,8.147892e-03,9.975158e-01,4.866202e-03,G,T,0.400560,0.599440,0.400560,-0.141657
dDocent_Contig_5718-59,2996,31962,8.23077,2.011740e+01,728,0,8,0.010989,999.0,2,...,140.480700,2.023372e-34,1.000000e+00,1.879979e-34,G,T,0.519444,0.480556,0.480556,-0.622423
dDocent_Contig_5962-32,46959,7975205,129.00800,5.281290e+03,728,0,0,0.000000,999.0,2,...,207.058900,3.541364e-58,1.000000e+00,3.464819e-58,T,C,0.429945,0.570055,0.429945,-0.751807


In [94]:
frq_df

,CHROM,POS,N_ALLELES,N_CHR,A1_FREQ,A2_FREQ
dDocent_Contig_10-75,dDocent_Contig_10,75,2,728,C:0.892857,G:0.107143
dDocent_Contig_20-67,dDocent_Contig_20,67,2,728,G:0.936813,C:0.0631868
dDocent_Contig_1545-58,dDocent_Contig_1545,58,2,728,G:0.255495,T:0.744505
dDocent_Contig_1548-37,dDocent_Contig_1548,37,2,728,A:0.913462,G:0.0865385
dDocent_Contig_4618-31,dDocent_Contig_4618,31,2,728,G:0.835165,A:0.164835
dDocent_Contig_4842-25,dDocent_Contig_4842,25,2,728,T:0.858516,C:0.141484
dDocent_Contig_5506-50,dDocent_Contig_5506,50,2,722,G:0.592798,T:0.407202
dDocent_Contig_5693-58,dDocent_Contig_5693,58,2,714,G:0.40056,T:0.59944
dDocent_Contig_5718-59,dDocent_Contig_5718,59,2,720,G:0.519444,T:0.480556
dDocent_Contig_5962-32,dDocent_Contig_5962,32,2,728,T:0.429945,C:0.570055


In [95]:
hardy

,CHROM,POS,OBS(HOM1/HET/HOM2),E(HOM1/HET/HOM2),ChiSq_HWE,P_HWE,P_HET_DEFICIT,P_HET_EXCESS
dDocent_Contig_10-75,dDocent_Contig_10,75,305/40/19,290.18/69.64/4.18,65.945980,4.826685e-11,4.826685e-11,1.000000e+00
dDocent_Contig_20-67,dDocent_Contig_20,67,340/2/22,319.45/43.09/1.45,330.997000,1.239780e-33,1.239780e-33,1.000000e+00
dDocent_Contig_1545-58,dDocent_Contig_1545,58,5/176/183,23.76/138.48/201.76,26.724590,3.078361e-08,1.000000e+00,1.612011e-08
dDocent_Contig_1548-37,dDocent_Contig_1548,37,301/63/0,303.73/57.55/2.73,3.266925,9.269385e-02,1.000000e+00,5.292059e-02
dDocent_Contig_4618-31,dDocent_Contig_4618,31,244/120/0,253.89/100.22/9.89,14.179360,1.169582e-05,1.000000e+00,7.427118e-06
dDocent_Contig_4842-25,dDocent_Contig_4842,25,267/91/6,268.29/88.43/7.29,0.308136,8.282703e-01,7.671565e-01,3.901890e-01
dDocent_Contig_5506-50,dDocent_Contig_5506,50,109/210/42,126.86/174.28/59.86,15.162130,1.276071e-04,9.999706e-01,7.408596e-05
dDocent_Contig_5693-58,dDocent_Contig_5693,58,45/196/116,57.28/171.44/128.28,7.326722,8.147892e-03,9.975158e-01,4.866202e-03
dDocent_Contig_5718-59,dDocent_Contig_5718,59,41/292/27,97.14/179.73/83.14,140.480700,2.023372e-34,1.000000e+00,1.879979e-34
dDocent_Contig_5962-32,dDocent_Contig_5962,32,0/313/51,67.29/178.43/118.29,207.058900,3.541364e-58,1.000000e+00,3.464819e-58


In [96]:
loci_out = os.path.join(analysis_dir,'loci_out.csv')
frq_out = os.path.join(analysis_dir,'frq_out.csv')
hardy_out = os.path.join(analysis_dir,'hardy_out.csv')

loci_df.to_csv(path_or_buf=loci_out)
frq_df.to_csv(path_or_buf=frq_out)
hardy.to_csv(path_or_buf=hardy_out)


In [97]:
loci_df.SUM_DEPTH.describe()

count    6.553000e+04
mean     7.833092e+03
std      3.629683e+04
min      5.920000e+02
25%      1.760000e+03
50%      2.946000e+03
75%      5.640750e+03
max      2.891540e+06
Name: SUM_DEPTH, dtype: float64

In [98]:
loci_df.QUAL.describe()

count    65530.00000
mean       878.38867
std        254.84026
min          3.04845
25%        999.00000
50%        999.00000
75%        999.00000
max        999.00000
Name: QUAL, dtype: float64

In [99]:
len(loci_df[loci_df.Fis == -9])

0

In [100]:
len(loci_df)

65530

In [101]:
len(loci_df[loci_df.SUM_DEPTH <10000]),len(loci_df[loci_df.SUM_DEPTH >=1000])

(56960, 62974)

In [102]:
len(loci_df[loci_df.QUAL >= 10]) - len(loci_df[loci_df.QUAL >= 20])

113

In [103]:
len(loci_df[loci_df.QUAL <  999]), len(loci_df[loci_df.QUAL < 500]), len(loci_df[loci_df.QUAL < 200])

(15784, 8054, 2947)

In [104]:
 len(loci_df[loci_df.Fis <= -0.5]), len(loci_df[loci_df.MAF < 0.05])

(4788, 180)

In [105]:
def filter_snps(df, imputed=False):
    if imputed:
        return df[(df.MAF >= 0.01) &  
                  (df.Fis > -0.5)]
    else:
        return df[(df.SUM_DEPTH >= 1000) & 
                  (df.SUM_DEPTH < 10000) & 
                  (df.QUAL >= 999) & 
                  (df.MAF >= 0.05) &  
                  (df.Fis > -0.5)]

In [106]:
loci_stage1 = filter_snps(loci_df)
loci_stage1.shape

(38103, 25)

In [107]:
with open(os.path.join(analysis_dir, "stage1_positions.txt"), "w") as o:
    for elem in loci_stage1.index:
        o.write("%s\n" % "\t".join(elem.split("-")))

In [108]:
!$vcftools --gzvcf $vcf_bad_remove_gz \
--remove-indels  \
--remove-filtered-all \
--recode \
--recode-INFO-all \
--positions {os.path.join(analysis_dir, "stage1_positions.txt")} \
--out {os.path.join(analysis_dir, "good_snps")}


VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf /home/faske/g/balsam/SNPcall/filtering/snps_indv_removed.recode.vcf.gz
	--recode-INFO-all
	--out /home/faske/g/balsam/SNPcall/filtering/good_snps
	--positions /home/faske/g/balsam/SNPcall/filtering/stage1_positions.txt
	--recode
	--remove-filtered-all
	--remove-indels

Using zlib version: 1.2.11
After filtering, kept 364 out of 364 Individuals
Outputting VCF file...
After filtering, kept 38103 out of a possible 65530 Sites
Run Time = 61.00 seconds


In [109]:
snps = os.path.join(analysis_dir, "good_snps.recode.vcf")
snps_gz = snps + ".gz"
!$bgzip -c {snps} > {snps_gz}
!$tabix {snps_gz}

# Plink / chrom_map

In [110]:
chroms = sorted(set([x.split("-")[0] for x in loci_df.index]))
len(chroms) # number of unique contigs

65530

In [111]:
with open(os.path.join(analysis_dir, "chrom_map.txt"), "w") as o:
    for i, c in enumerate(chroms):
        o.write("%s\t%d\n" % (c, i))

In [112]:
def write_plink_files(vcf_gz):
    !$vcftools --gzvcf {vcf_gz} \
    --out {vcf_gz} \
    --plink \
    --chrom-map {os.path.join(analysis_dir, "chrom_map.txt")}

In [113]:
write_plink_files(snps_gz)


VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf /home/faske/g/balsam/SNPcall/filtering/good_snps.recode.vcf.gz
	--chrom-map /home/faske/g/balsam/SNPcall/filtering/chrom_map.txt
	--out /home/faske/g/balsam/SNPcall/filtering/good_snps.recode.vcf.gz
	--plink

Using zlib version: 1.2.11
After filtering, kept 364 out of 364 Individuals
Writing PLINK PED and MAP files ... 
	Read 65531 chromosome mapping file entries.
Done.
After filtering, kept 38103 out of a possible 38103 Sites
Run Time = 7.00 seconds


# Make 012

In [114]:
f = os.path.join(analysis_dir, "good_snps.recode.vcf")
!$vcftools --gzvcf {f} \
--out {f} \
--012


VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf /home/faske/g/balsam/SNPcall/filtering/good_snps.recode.vcf
	--012
	--out /home/faske/g/balsam/SNPcall/filtering/good_snps.recode.vcf

Using zlib version: 1.2.11
After filtering, kept 364 out of 364 Individuals
Writing 012 matrix files ... Done.
After filtering, kept 38103 out of a possible 38103 Sites
Run Time = 4.00 seconds


# Get coverage per individual

In [115]:
f = os.path.join(analysis_dir, "good_snps.recode.vcf")
!$vcftools --gzvcf {f} \
--out {f} \
--depth


VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--gzvcf /home/faske/g/balsam/SNPcall/filtering/good_snps.recode.vcf
	--depth
	--out /home/faske/g/balsam/SNPcall/filtering/good_snps.recode.vcf

Using zlib version: 1.2.11
After filtering, kept 364 out of 364 Individuals
Outputting Mean Depth by Individual
After filtering, kept 38103 out of a possible 38103 Sites
Run Time = 3.00 seconds


In [116]:
depth_file = os.path.join(analysis_dir, "good_snps.recode.vcf.idepth")
depth_df = pd.read_csv(depth_file, sep="\t")
depth_df.head()

,INDV,N_SITES,MEAN_DEPTH
0,/home/faske/g/balsam/SNPcall/bowtie2/BS_AN_10....,38103,9.05168
1,/home/faske/g/balsam/SNPcall/bowtie2/BS_AN_11....,38103,8.70178
2,/home/faske/g/balsam/SNPcall/bowtie2/BS_AN_12....,38103,8.55817
3,/home/faske/g/balsam/SNPcall/bowtie2/BS_AN_1.F...,38103,9.29378
4,/home/faske/g/balsam/SNPcall/bowtie2/BS_AN_2.F...,38103,12.45950


In [117]:
depth_df.MEAN_DEPTH.describe()

count    364.000000
mean       8.829000
std        1.779591
min        3.070490
25%        7.788625
50%        8.909010
75%        9.949100
max       14.113200
Name: MEAN_DEPTH, dtype: float64

# rm intermediate files

In [118]:
!rm snps*

In [119]:
!rm BASA*

In [120]:
!rm *vcf